### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [10]:
import numpy as np

In [11]:
class KnnBruteClassifier:
    """Классификатор реализует взвешенное голосование по ближайшим соседям.
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    """
    def __init__(self, n_neighbors=1, weights='uniform', metric="l2"):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric


    def fit(self, X, y):
        """Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        predictions = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            if self.weights == 'uniform':
                nearest_indices = np.argpartition(distances, self.n_neighbors)[:self.n_neighbors]
            elif self.weights == 'distance':
                inverse_distances = 1 / (distances + 1e-10)
                weighted_nearest_indices = np.argpartition(inverse_distances, self.n_neighbors)[self.n_neighbors:]
                nearest_indices = weighted_nearest_indices[np.argsort(inverse_distances[weighted_nearest_indices])]

            nearest_labels = self.y_train[nearest_indices]
            unique, counts = np.unique(nearest_labels, return_counts=True)
            predicted_label = unique[np.argmax(counts)]
            predictions.append(predicted_label)
        return np.array(predictions)

    def predict_proba(self, X):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности
        объекта к каждому классу
        """
        proba = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            if self.weights == 'uniform':
                nearest_indices = np.argpartition(distances, self.n_neighbors)[:self.n_neighbors]
            elif self.weights == 'distance':
                inverse_distances = 1 / (distances + 1e-10)
                weighted_nearest_indices = np.argpartition(inverse_distances, self.n_neighbors)[self.n_neighbors:]
                nearest_indices = weighted_nearest_indices[np.argsort(inverse_distances[weighted_nearest_indices])]

            nearest_labels = self.y_train[nearest_indices]
            unique, counts = np.unique(nearest_labels, return_counts=True)
            class_proba = counts / self.n_neighbors
            proba.append(class_proba)
        return np.array(proba)

    def kneighbors(self, X, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        neigh_dist = []
        neigh_ind = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            nearest_indices = np.argpartition(distances, n_neighbors)[:n_neighbors]
            nearest_distances = distances[nearest_indices]
            neigh_dist.append(nearest_distances)
            neigh_ind.append(nearest_indices)
        return np.array(neigh_dist), np.array(neigh_ind)

In [12]:
def load_file(filename):
    raw_data = np.load(filename, allow_pickle=True)
    return {'X_train': raw_data[()]['X_train'], 'y_train': raw_data[()]['y_train'], 'X_test': raw_data[()]['X_test']}

In [13]:
input_filename = "knn_data_065.npy"
data_dict = load_file(input_filename)

In [14]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [15]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [16]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [17]:
output_filename = "results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)

In [2]:
from z3 import *
s=Solver()
x=BitVec("x" ,32)
a=1103515245
c=12345
s.add((((x*a)+c)>>16) & 32767 == 4583)
s.add((((((x*a)+c)*a)+c)>>16) & 32767 == 16304)
s.check()
print(s.model())

[x = 11223344]
